In [1]:
import duckdb
import yaml
from mosaic_widget import MosaicWidget
import ipywidgets as widgets
import pyarrow as pa

In [2]:
# Optional: enable logging in Mosaic widget
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
def load_arrow(con, path, table):
    with pa.memory_map(path) as source:
        my_arrow = pa.ipc.open_stream(source).read_all()
        con.execute(f"CREATE TABLE {table} AS FROM my_arrow")
        
def load_csv(con, path, table):
    con.execute(f"CREATE TABLE {table} AS FROM read_csv_auto('{path}')")

In [4]:
con = duckdb.connect()
load_arrow(con, "../../data/flights-200k.arrow", "flights")
load_arrow(con, "../../data/random-walk.arrow", "walk")
load_csv(con, "../../data/penguins.csv", "penguins")
load_csv(con, "../../data/athletes.csv", "athletes")
load_csv(con, "../../data/seattle-weather.csv", "weather")

In [5]:
mosaic = MosaicWidget(con=con)

In [6]:
dropdown = widgets.Dropdown(
    options=[("Athletes", "athletes"), ("Cross-Filter", "cross-filter"), ("Density 1D", "density1d"), ("Facets", "facets"),
             ("Flights 10M", "flights-10m"), ("Gaia Star Catalog", "gaia"), ("Hexbin", "hexbin"), ("Mark Gallery", "marks"),
             ("Line Density", "line-density"), ("Pan + Zoom", "pan-zoom"), ("Regression", "regression"), ("Seattle Weather", "weather")],
    value="weather",
    description="Spec:"
)

def on_change(change):
    open_spec(change["new"])

def open_spec(spec):
    with open(f"../specs/{spec}.yaml") as f:
        mosaic.spec = yaml.safe_load(f)

dropdown.observe(on_change, "value")

open_spec("weather")

In [8]:
widgets.VBox([dropdown, mosaic])